
http://www.tennis-data.co.uk/2022/2022.xlsx
http://www.tennis-data.co.uk/2021/2021.xlsx

In [41]:
import numpy as np
import pandas as pd
import os
import pickle 
import procyclingstats

def rolling_resistance():
    return 0.0050

def gravity(slope, weight):
    return 9.80665 * np.sin(np.arctan(slope)) * (weight + 6.8)

def aerodynamic_drag(v):
    cda = 0.3
    elevation = 375
    p = (1.225* np.exp(-0.00011856*elevation))
    w = 0
    return 0.5 * cda * p * (v + w)**2

def cycling_power(slope, weight, velocity):
    Fg = gravity(slope, weight)
    Fr = rolling_resistance()
    Fa = aerodynamic_drag(velocity)
    return ((Fg + Fr + Fa) * velocity) / (1-0.03)



In [42]:
from procyclingstats import Race

race = Race("race/tour-de-france/2022")

In [91]:
df = pd.DataFrame({"year":[0],"Url": [""]})
for year in range(1958,2024):
    race = Race("race/tour-de-france/" + str(year))
    stages = []
    for stage in race.stages():
        print(f"Stage {stage['stage_url']}.")
        stages.append(stage['stage_url'])
    df = pd.concat([df, pd.DataFrame({'year': [year]*len(stages), 'Url': stages})])


Stage race/tour-de-france/2023/stage-1.
Stage race/tour-de-france/2023/stage-2.
Stage race/tour-de-france/2023/stage-3.
Stage race/tour-de-france/2023/stage-4.
Stage race/tour-de-france/2023/stage-5.
Stage race/tour-de-france/2023/stage-6.
Stage race/tour-de-france/2023/stage-7.
Stage race/tour-de-france/2023/stage-8.
Stage race/tour-de-france/2023/stage-9.
Stage race/tour-de-france/2023/stage-10.
Stage race/tour-de-france/2023/stage-11.
Stage race/tour-de-france/2023/stage-12.
Stage race/tour-de-france/2023/stage-13.
Stage race/tour-de-france/2023/stage-14.
Stage race/tour-de-france/2023/stage-15.
Stage race/tour-de-france/2023/stage-16.
Stage race/tour-de-france/2023/stage-17.
Stage race/tour-de-france/2023/stage-18.
Stage race/tour-de-france/2023/stage-19.
Stage race/tour-de-france/2023/stage-20.
Stage race/tour-de-france/2023/stage-21.


In [197]:
df.iloc[-21:].Url

0      race/tour-de-france/2023/stage-1
1      race/tour-de-france/2023/stage-2
2      race/tour-de-france/2023/stage-3
3      race/tour-de-france/2023/stage-4
4      race/tour-de-france/2023/stage-5
5      race/tour-de-france/2023/stage-6
6      race/tour-de-france/2023/stage-7
7      race/tour-de-france/2023/stage-8
8      race/tour-de-france/2023/stage-9
9     race/tour-de-france/2023/stage-10
10    race/tour-de-france/2023/stage-11
11    race/tour-de-france/2023/stage-12
12    race/tour-de-france/2023/stage-13
13    race/tour-de-france/2023/stage-14
14    race/tour-de-france/2023/stage-15
15    race/tour-de-france/2023/stage-16
16    race/tour-de-france/2023/stage-17
17    race/tour-de-france/2023/stage-18
18    race/tour-de-france/2023/stage-19
19    race/tour-de-france/2023/stage-20
20    race/tour-de-france/2023/stage-21
Name: Url, dtype: object

In [201]:
def to_seconds(time):
    h, m, s = time.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

In [306]:
stage.stage_type()

'RR'

In [300]:
from procyclingstats import Stage, Rider
powers = []
stages = []

for url in df.iloc[1:].Url:
    stage = Stage(url)
    if not stage.stage_type() == "TTT":
        rider = Rider(stage.results('rider_url')[0]['rider_url'])
        stages.append(stage)

        if stage.vertical_meters() == None:
            grade = 1000 / (stage.distance() * 1000)
        else:
            grade = stage.vertical_meters() / (stage.distance() * 1000)
        
        total_seconds = to_seconds(stage.results('time')[0]['time'])
    
        velocity = (stage.distance()*1000) / total_seconds
        
        try:
            print(cycling_power(grade,rider.weight(),velocity))
            powers.append(cycling_power(grade,rider.weight(),velocity))
        except IndexError as err:
            print(cycling_power(grade,70,velocity))
            powers.append(cycling_power(grade,70,velocity))
            
    
    

315.3027231302158
237.27848897112045
251.655797711142
345.38865250042727
362.2262417620344
373.77835301562817
376.43276192681606
370.8392360413004
387.48374095408997
332.2836288602003
380.77251240402876
261.0254871255787
333.83605447403676
358.4937603457267
330.137413095782
205.6183021068715
311.5621124118013
346.23409314810596
350.3750795993483
321.57721910950295
297.65879060570273
254.67076976883703
402.54341279599635
212.05669137618784
343.03533430499954
221.95929382718072
202.69513729830237
191.1133909230663
196.9681582472929
645.854388893753
366.5706470103642
314.2649807398686
269.1853660897206
225.85684470069793
242.94975867121494
327.69034886428557
205.5289335899173
166.72062710961376
386.21185287292343
169.23983820065646
202.59663813257734
144.54408377419622
123.20243961313078
194.8366181142102
408.5585054302662
165.5555050708469
292.93671144345365
347.60578631680465
288.67267741527763
345.4633090352157
349.52823612571666
413.3486916051314
443.79493536066144
336.2104704347651
3

UnexpectedParsingError: Field 'rider_url' wasn't parsed correctly

In [308]:
for stage, power in zip(stages, powers):
    print(stage.departure() + "-" + stage.arrival() + ":" + str(stage.distance()) + " power: "+ str(power))

Brussel-Ghent:184.0 power: 315.3027231302158
Ghent-Dunkirk:198.0 power: 237.27848897112045
Dunkirk-Mers-les-Bains:177.0 power: 251.655797711142
Le Tréport-Versailles:205.0 power: 345.38865250042727
Versailles-Caen:232.0 power: 362.2262417620344
Caen-Saint-Brieuc:223.0 power: 373.77835301562817
Saint-Brieuc-Brest:170.0 power: 376.43276192681606
Chateaulin-Chateaulin:46.0 power: 370.8392360413004
Quimper-Saint-Nazaire:206.0 power: 387.48374095408997
Saint-Nazaire-Royan:255.0 power: 332.2836288602003
Royan-Bordeaux:137.0 power: 380.77251240402876
Bordeaux-Dax:161.0 power: 261.0254871255787
Dax-Pau:230.0 power: 333.83605447403676
Pau-Luchon:129.0 power: 358.4937603457267
Luchon-Toulouse:176.0 power: 330.137413095782
Toulouse-Béziers:187.0 power: 205.6183021068715
Béziers-Nîmes:189.0 power: 311.5621124118013
Mont Ventoux-Mont Ventoux:21.0 power: 346.23409314810596
Carpentras-Gap:178.0 power: 350.3750795993483
Gap-Briançon:165.0 power: 321.57721910950295
Briançon-Aix-les-Bains:219.0 power: 2

412.43443874898816

In [192]:


print(cycling_power(grade,rider.weight(),velocity))

420.72490011526565


In [183]:
print(cycling_power(grade,rider.weight(),velocity))


69.8692650829908


In [181]:
grade

0.005013032145960035